In [19]:
import pandas as pd
import numpy as np
import re
from calendar import monthrange
from datetime import datetime, timedelta

In [21]:
file_path = '../dataset/naver_datalab_copy.xlsx'
sheet_name = 'Sheet1'
df = pd.read_excel(file_path, sheet_name=sheet_name, header=[0, 1, 2])

# 각 열의 이름을 연결하여 단일 헤더로 만드는 작업
# 주의: 실제 적용할 때 공백과 NaN을 적절히 처리 필요
df.columns = [' '.join(map(str, col)).strip() for col in df.columns]

# 지정된 컬럼 이름을 변경합니다.
df.rename(columns={
    '합계: 순위 Unnamed: 0_level_1 구분연간순위': '구분연간순위',
    '합계: 순위 Unnamed: 1_level_1 Sort.구분': 'Sort.구분',
    '합계: 순위 Unnamed: 2_level_1 키워드연간순위': '키워드연간순위',
    '합계: 순위 Unnamed: 3_level_1 상품명': '상품명',
    '월(연도) 2017 08월': '2017 08월'
}, inplace=True)

# '월(월)'로 시작하는 컬럼명에서 '월(월)'을 제거
df.columns = [col.replace('월(월) ', '') if col.startswith('월(월)') else col for col in df.columns]
df

,구분연간순위,Sort.구분,키워드연간순위,상품명,2017 08월,2017 09월,2017 10월,2017 11월,2017 12월,2018 01월,...,2024 03월,2024 04월,2024 05월,2024 06월,2024 07월,2024 08월,2024 09월,2024 10월,2024 11월,2024 12월
0,1,타이어,23,2355519,59.0,55.0,37.0,41.0,43.0,54.0,...,42.0,47.0,48.0,53.0,41.0,55.0,62.0,49.0,54.0,32.0
1,1,타이어,31,2454518,49.0,45.0,33.0,38.0,51.0,56.0,...,52.0,56.0,63.0,64.0,59.0,77.0,85.0,75.0,76.0,42.0
2,1,타이어,48,타이어,24.0,23.0,20.0,23.0,28.0,36.0,...,65.0,75.0,70.0,68.0,65.0,109.0,111.0,111.0,154.0,118.0
3,1,타이어,52,2255517,103.0,79.0,57.0,59.0,78.0,103.0,...,80.0,96.0,94.0,84.0,90.0,117.0,110.0,106.0,123.0,86.0
4,1,타이어,58,2356018,108.0,91.0,63.0,57.0,75.0,105.0,...,91.0,112.0,116.0,98.0,107.0,128.0,121.0,121.0,125.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,44,스노우체인,536,타이어체인,NaN,NaN,NaN,192.0,36.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,67.0
621,46,포켓,359,차량용사이드포켓,NaN,NaN,NaN,NaN,NaN,90.0,...,439.0,403.0,322.0,NaN,NaN,NaN,NaN,NaN,371.0,NaN
622,49,LED등,701,브라비오h4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
623,49,LED등,728,led전조등,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132.0,399.0


In [29]:
# '연 월' 형식 컬럼들만 추출
date_columns = [col for col in df.columns if '월' in col and '상품명' not in col]

# 결과 저장을 위한 새로운 DataFrame
weekly_data = pd.DataFrame()

# 각 월별 데이터에 대해 주별 데이터로 나누기
for col in date_columns:
    year, month_str = col.split()
    # '월' 제거하고 정수형 변환
    month = int(month_str.replace('월', ''))
    year = int(year)
    
    days_in_month = monthrange(year, month)[1]  # 해당 월의 일 수
    
    # 월의 첫 번째 날
    start_date = datetime(year, month, 1)
    
    # 주별 데이터 계산
    week_start = start_date
    while week_start <= datetime(year, month, days_in_month):
        week_end = min(week_start + timedelta(days=6), datetime(year, month, days_in_month))
        days_in_week = (week_end - week_start).days + 1
        
        # 날짜 범위로 새로운 컬럼 이름 생성
        week_col = f'{week_start.strftime("%Y-%m-%d")} - {week_end.strftime("%Y-%m-%d")}'
        
        # 주별 데이터 저장
        weekly_data[week_col] = (df[col] * (days_in_week / days_in_month)).round(2)

        # 다음 주 시작일
        week_start = week_end + timedelta(days=1)

# '상품명'과 함께 주별 데이터 합치기
weekly_data = pd.concat([df['상품명'], weekly_data], axis=1)

weekly_data

C:\Users\hyejinbaek\AppData\Local\Temp\ipykernel_24740\2875343678.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weekly_data[week_col] = (df[col] * (days_in_week / days_in_month)).round(2)
C:\Users\hyejinbaek\AppData\Local\Temp\ipykernel_24740\2875343678.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weekly_data[week_col] = (df[col] * (days_in_week / days_in_month)).round(2)
C:\Users\hyejinbaek\AppData\Local\Temp\ipykernel_24740\2875343678.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

,상품명,2017-08-01 - 2017-08-07,2017-08-08 - 2017-08-14,2017-08-15 - 2017-08-21,2017-08-22 - 2017-08-28,2017-08-29 - 2017-08-31,2017-09-01 - 2017-09-07,2017-09-08 - 2017-09-14,2017-09-15 - 2017-09-21,2017-09-22 - 2017-09-28,...,2024-11-01 - 2024-11-07,2024-11-08 - 2024-11-14,2024-11-15 - 2024-11-21,2024-11-22 - 2024-11-28,2024-11-29 - 2024-11-30,2024-12-01 - 2024-12-07,2024-12-08 - 2024-12-14,2024-12-15 - 2024-12-21,2024-12-22 - 2024-12-28,2024-12-29 - 2024-12-31
0,2355519,13.32,13.32,13.32,13.32,5.71,12.83,12.83,12.83,12.83,...,12.60,12.60,12.60,12.60,3.60,7.23,7.23,7.23,7.23,3.10
1,2454518,11.06,11.06,11.06,11.06,4.74,10.50,10.50,10.50,10.50,...,17.73,17.73,17.73,17.73,5.07,9.48,9.48,9.48,9.48,4.06
2,타이어,5.42,5.42,5.42,5.42,2.32,5.37,5.37,5.37,5.37,...,35.93,35.93,35.93,35.93,10.27,26.65,26.65,26.65,26.65,11.42
3,2255517,23.26,23.26,23.26,23.26,9.97,18.43,18.43,18.43,18.43,...,28.70,28.70,28.70,28.70,8.20,19.42,19.42,19.42,19.42,8.32
4,2356018,24.39,24.39,24.39,24.39,10.45,21.23,21.23,21.23,21.23,...,29.17,29.17,29.17,29.17,8.33,19.65,19.65,19.65,19.65,8.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,타이어체인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42.00,42.00,42.00,42.00,12.00,15.13,15.13,15.13,15.13,6.48
621,차량용사이드포켓,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.57,86.57,86.57,86.57,24.73,NaN,NaN,NaN,NaN,NaN
622,브라비오h4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
623,led전조등,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.80,30.80,30.80,30.80,8.80,90.10,90.10,90.10,90.10,38.61


In [31]:
# 주별 데이터 엑셀 파일로 저장
output_file_path = '../dataset/weekly_data.xlsx'
weekly_data.to_excel(output_file_path, index=False, engine='openpyxl')
print(f"Data has been successfully saved to {output_file_path}")

Data has been successfully saved to ../dataset/weekly_data.xlsx
